# Segmenting and clustering neighborhoods in Toronto

This notebook reads wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) to get a list of boroughs and neighborhoods of Toronto.  The data will be loaded into a pandas dataframe.

Assignment instructions:
1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood (in step 2)
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned** (in step 3.1)
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table. (in step 3.2)
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park (in step 3.3)
5. Finally, in the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## Step 0 - import libraries

In [1]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

print("Libraries imported.")

Libraries imported.


## Step 1

Download the wiki page, parse it and explore the data a bit to understand its structure.


In [2]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Download the wiki page and parse it in.
wiki_bs = BeautifulSoup(requests.get(wiki_url).text)
print("Downloading data ok!")

Next, examine the data we have...

In [3]:
# from examining the wiki page visually we know that our desired data is in the first table on the page
w_postal_code_rows = wiki_bs.find('table').find_all('tr')

# Confirm with some exploring that we have found correct data
print("\n\nTotal of", len(w_postal_code_rows), "postal code rows found\n")
print("Header row of the table:")
print(w_postal_code_rows[0])
print("\nfirst three rows of data:")
print(w_postal_code_rows[1:4])



Total of 290 postal code rows found

Header row of the table:
<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

first three rows of data:
[<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>]


#### Step 1 results:

Above printout of data gives us a good understanding of what we are dealing with:
- There are 290 rows in the HTML table, of those 1 is header row and 289 are data rows.
- Each datarow's internal structure seems like quite basic html with **tr**, **td** and some **a** tags.
- also there appears to be a newline character always in the 3rd column, the _neighbourhood_ column.

## Step 2

Create a dataframe and parse the above data into it.

In [4]:

# First define a helper function to read the data from BeautifulSoup into a dataframe

# returns parsed strings: postcode, borough, neighbourhood
def parse_wiki_postal_code_row(bs_tr):
    if not isinstance(bs_tr, bs4.element.Tag):
        print("parse_wiki_postal_code_row: Error, wrong type of input", bs_tr)
        return False
    cols = bs_tr.find_all('td')
    if len(cols) != 3:
        print("Something wrong with this row? : ", bs_tr)
        return False

    # parse out the data
    # 3rd column seems to include a newline char, remove it on the go...
    postcode = cols[0].get_text()
    borough = cols[1].get_text()
    neighbourhood = cols[2].get_text().replace('\n', '')

    return postcode, borough, neighbourhood

print("helper function defined.\n")


# Create the dataframe
toronto_df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

# ... and fill it up
for row in w_postal_code_rows[1:]:
    p, b, n = parse_wiki_postal_code_row(row)
    row_data = {'PostalCode': p, 'Borough': b, 'Neighborhood': n}
    toronto_df = toronto_df.append(row_data, ignore_index=True)

print("toronto_df has", toronto_df.shape[0], "rows of data.\n")
toronto_df.head()

helper function defined.

toronto_df has 289 rows of data.



,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# another quick peek into the data
toronto_df.describe()

,PostalCode,Borough,Neighborhood
count,289,289,289
unique,180,12,210
top,M9V,Not assigned,Not assigned
freq,8,77,78


#### Step 2 results:

As seen above in step 1, there are 289 data rows, and reading the data into *toronto_df* dataframe appears to have succeeded.

## Step 3 - Clean the data


### Cleanup #1

_Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned._

In [6]:
# remove rows whose Borough is "Not assigned"
toronto_df = toronto_df[toronto_df["Borough"] != "Not assigned"]
print("\n\nNow toronto_df has",toronto_df.shape[0], "rows of data.\n")
toronto_df.head()



Now toronto_df has 212 rows of data.



,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Cleanup #2

_More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma._

First find out the postal codes that are mentioned in *toronto_df* more than once.

In [7]:
g_toronto_df = toronto_df.groupby("PostalCode").count()
doubles_df = g_toronto_df[g_toronto_df['Borough'] > 1]

print("\n\nFound", doubles_df.shape[0], "postal codes to appear more than once in the wikipedia data.\n")
doubles_df.head()



Found 58 postal codes to appear more than once in the wikipedia data.



,Borough,Neighborhood
PostalCode,,
M1B,2,2
M1C,3,3
M1E,3,3
M1K,3,3
M1L,3,3


Then, for each postal code identified above:
- find all different neighborhoods that belong to the postal code in the *toronto_df* and join them into one string,
- check that the **borough** is the same for all rows of data
- remove the old postal code entries from the *toronto_df*
- add new entry to the *toronto_df*, where the postal code is only once and all neighborhoods are in one string, comma separated.

In [8]:

# loop to merge postal codes with many rows (neighborhoods) into one row

for index, row in doubles_df.iterrows():
    filter = toronto_df['PostalCode'] == index
    neighborhoods_Series = toronto_df[filter]['Neighborhood']
    boroughs_Series = toronto_df[filter]['Borough']

    # Avoid running this cleanup more than once, it could happend if this cell
    # was executed multiple times consequently...
    if len(neighborhoods_Series) > 1:
        # concatenate the neighborhoods into one string
        s = ", ".join(neighborhoods_Series.sort_values())

        # ensure all postal codes belong to same borough
        if len(set(boroughs_Series)) > 1:
            print("*************: check failed for", index)
        borough = boroughs_Series.iloc[0]

        # drop old rows of the postal code from the toronto dataset
        toronto_df = toronto_df[toronto_df['PostalCode'] != index]    

        # add new row with merged neighborhood data
        row_data = {'PostalCode': index, 'Borough': borough, 'Neighborhood': s}
        toronto_df = toronto_df.append(row_data, ignore_index=True)

print("\n\nNow toronto_df has", toronto_df.shape[0], "rows of data.\n")
toronto_df.head()



Now toronto_df has 103 rows of data.



,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M7A,Queen's Park,Not assigned
3,M9A,Etobicoke,Islington Avenue
4,M3B,North York,Don Mills North


### Cleanup #3

_If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park._

So first find out how many such cases there are.

In [9]:
filter = toronto_df['Neighborhood'].str.contains("Not assigned")
toronto_df[filter]

,PostalCode,Borough,Neighborhood
2,M7A,Queen's Park,Not assigned


Just one, ok.  Still we clean it in a way that would handle many rows as well.

In [10]:
# assign neighborhood values from borough
toronto_df.loc[filter, 'Neighborhood'] = toronto_df.loc[filter, 'Borough']

# show result
toronto_df[filter]

,PostalCode,Borough,Neighborhood
2,M7A,Queen's Park,Queen's Park


Cleaning is done successfully.

## Final result

First write the cleaned data into a file, from where we can load it in the next phase of this assignment.

In [11]:

# save data to file
output_file_name = "toronto_postal_cleaned.csv"
text_file = open(output_file_name, 'w')
text_file.write(toronto_df.to_csv(index=False))
text_file.close()
print("toronto_df written to file", output_file_name)

toronto_df written to file toronto_postal_cleaned.csv



Finally, let's see how the data looks now, and finish up with the *shape* information in the last cell.



In [12]:
toronto_df.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,11,103
top,M4A,North York,Woodbine Heights
freq,1,24,1


In [13]:
toronto_df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M7A,Queen's Park,Queen's Park
3,M9A,Etobicoke,Islington Avenue
4,M3B,North York,Don Mills North
5,M6B,North York,Glencairn
6,M4C,East York,Woodbine Heights
7,M5C,Downtown Toronto,St. James Town
8,M6C,York,Humewood-Cedarvale
9,M4E,East Toronto,The Beaches


In [14]:
toronto_df.shape

(103, 3)